In [20]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from config import *

In [21]:
client = QdrantClient(URL,port=6333)

2024-02-29 03:35:19.264 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


In [22]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [23]:
from llama_index.llms.azure_openai import AzureOpenAI
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name=deployment_id_gpt4,
    api_key=key,
    azure_endpoint=endpoint,
    api_version=api_version,
)

In [24]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding()

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 18771.13it/s]


In [25]:
from llama_index.core import Settings
from llama_index.core  import VectorStoreIndex

Settings.llm = llm
Settings.embed_model = embed_model

In [26]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [27]:
query_engine = index.as_query_engine()

In [28]:
query = "What is  ENERGY INTENSITY AT TSJ?"

In [29]:
response = query_engine.query(query)

In [30]:
print(response.get_formatted_sources())
print("query was:", query)
print("answer was:", response)

> Source (Doc id: dec6495a-e54f-4d9a-87a8-98082f401c6b): Further, 
we aim to achieve CO2 emission 
intensity of <1.8 tCO2/tcs for Tata 
Steel India by 203...

> Source (Doc id: bca0538f-8954-4843-8c84-29d265eae814): Asia’s first integrated steel plant
Tata Steel Jamshedpur (TSJ) is our flagship 
facility. From d...
query was: What is  ENERGY INTENSITY AT TSJ?
answer was: The given excerpt does not provide information on the ENERGY INTENSITY AT TSJ.


In [31]:
from pprint import pprint

In [32]:
pprint(str(response))

'The given excerpt does not provide information on the ENERGY INTENSITY AT TSJ.'


In [33]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

In [34]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
    llm = llm,
)

In [35]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

In [36]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query(query)

In [37]:
pprint(str(response))

('The energy intensity at TSJ is 5.61 Gcal/tcs, according to the excerpt from '
 'the document titled "Innovative Approaches to Sustainability and Steel '
 'Production for Building Landmarks and Advancing Industries."')
